In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import pandas_datareader.data as web
from datetime import datetime, timedelta
import scipy.stats as stats
from sklearn.metrics import brier_score_loss, roc_curve, auc, log_loss
from sklearn.preprocessing import StandardScaler

/Users/xiaozhezhang/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


| market_category | feature_name | id |
|-----------------|--------------|----|
| Bank            | bac          |  1 |
| Bank            | citi         |  2 |
| Commodity       | corn         |  3 |
| Currency        | euro         |  4 |
| Commodity       | gold         |  5 |
| Inflation       | infl5y       |  6 |
| Commodity       | iyr          |  7 |
| Currency        | pound        |  8 |
| Commodity       | silver       |  9 |
| Commodity       | soybns       | 10 |
| Equity          | sp12m        | 11 |
| Equity          | sp6m         | 12 |
| Commodity       | wheat        | 13 |
| Currency        | yen          | 14 |


In [2]:
df = pd.read_csv("mpd_sp500.csv")
df.head()

,Date,f1_maturity_target,f1_mu,f1_sd,f1_skew,f1_kurt,f1_p10,f1_p50,f1_p90,f1_lg_change_decr,...,f14_p50,f14_p90,f14_lg_change_decr,f14_prDec,f14_lg_change_incr,f14_prInc,SP_adj_close,SP_lg_pr,SP_lg_ret(%),VIX
0,2013-01-17,3.0,-0.00364,0.15922,-0.25881,0.50275,-0.20880,0.00419,0.18737,-20.0,...,0.00280,0.06621,-10.0,0.03761,10.0,0.02347,1480.939941,7.300432,0.597345,0.1357
1,2013-01-24,3.0,-0.00801,0.15337,-0.57865,1.03350,-0.21009,0.00958,0.16336,-20.0,...,0.00258,0.06933,-10.0,0.04333,10.0,0.03237,1494.819946,7.309761,0.932878,0.1269
2,2013-01-31,3.0,-0.00801,0.15337,-0.57865,1.03350,-0.21009,0.00958,0.16336,-20.0,...,0.00258,0.06933,-10.0,0.04333,10.0,0.03237,1498.109985,7.311960,0.219854,0.1428
3,2013-02-07,3.0,-0.00430,0.15325,-0.34592,0.64703,-0.19913,0.00299,0.18027,-20.0,...,0.00383,0.07824,-10.0,0.05847,10.0,0.05089,1509.390015,7.319461,0.750130,0.1350
4,2013-02-14,3.0,-0.00430,0.15325,-0.34592,0.64703,-0.19913,0.00299,0.18027,-20.0,...,0.00383,0.07824,-10.0,0.05847,10.0,0.05089,1521.380005,7.327373,0.791222,0.1266


In [ ]:
df = pd.read_csv("mpd_sp500.csv")

# keep columns that contain f11 and f12 only, keep Date
df = df.filter(regex='f11|f12|Date')

# remove columns that contain matyrity_target
df = df.loc[:, ~df.columns.str.contains('maturity_target')]
df.head()

In [ ]:
sp = pd.read_csv("^SPX.csv")
sp.head()

In [ ]:
#drop Open, High, Low, Close, Volume
sp = sp.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis=1)
sp['log_price'] = np.log(sp['Adj Close'])
# sp['log_ret'] = np.log(sp['Adj Close'] / sp['Adj Close'].shift(1))
sp.head()

In [ ]:
# merge sp and df
df['Date'] = pd.to_datetime(df['Date'])
sp['Date'] = pd.to_datetime(sp['Date'])
mpd_sp = pd.merge(df, sp, on='Date', how='left')
# fill missing values with the previous value
mpd_sp = mpd_sp.fillna(method='ffill')
# remove Adj Close
mpd_sp = mpd_sp.drop(['Adj Close'], axis=1)
# rename log_price to sp_log_price
mpd_sp = mpd_sp.rename(columns={'log_price': 'sp_log_price'})
mpd_sp['sp_log_ret'] = mpd_sp['sp_log_price'] - mpd_sp['sp_log_price'].shift(1)


In [ ]:
mpd_sp.head()